In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, chi2, mutual_info_classif
import matplotlib.pyplot as plt

## Dados escolares

In [2]:
#abrindo os dados
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')
#agrupando por escolas
dados_escolares = dados_escolares.groupby('CODMEC').mean()
#Apagando a classe desbalanceada
nota = nota[nota.Classificacao != 0]
#Em todos Dataframes temos mais escolas do que o nosso target
dados_escolares = dados_escolares.reset_index()[dados_escolares.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
dados_escolares.drop(dados_escolares.columns[dados_escolares.isnull().mean() != 0], axis=1, inplace = True)
#Alterando o nome da coluna para o merge
nota.rename(columns = {'CODIGO DA ENTIDADE':'CODMEC'}, inplace = True)
#criando o merge com base na coluna CODMEC usando o dataframe target e os dados escolares
new_df = pd.merge(dados_escolares, nota, on = 'CODMEC')
new_df.drop(['CODMEC', 'ANO','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
#criando o data e o target
data = new_df.drop('Classificacao', axis = 1)
target = new_df.Classificacao
print(data.shape)

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6,8,11,17,18,19,20,111,240) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(692, 210)


### f regression

In [7]:
#Rodando o selectKBest para 12 colunas
melhor_score = 0
melhores_colunas = []
for var in list(range(1,45)):
    kbest = SelectKBest(f_regression, k=var)
    new_data = kbest.fit_transform(data, target)
    print(new_data.shape)
    lm = LinearRegression()
    scaler = StandardScaler()
    data1 = scaler.fit_transform(new_data)
    kf = KFold(5, shuffle = True)
    r = []
    for train_i, test_i in kf.split(data1):
        x_train, x_test = data1[train_i], data1[test_i]
        y_train, y_test = target[train_i], target[test_i]
        lm.fit(x_train, y_train)
        r.append(lm.score(x_test, y_test))
    print('Score:',sum(r)/5)
    if melhor_score < sum(r)/5:
        melhor_score = sum(r)/5
        melhores_colunas = data.columns[kbest.get_support()]

C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWar

(692, 1)
Score: 0.2080305111090956
(692, 2)
Score: 0.23659672378726687
(692, 3)
Score: 0.23994543902087656
(692, 4)
Score: 0.24517538886081286
(692, 5)
Score: 0.22209801684643588
(692, 6)
Score: 0.21281386247685868
(692, 7)
Score: 0.22631335764156524
(692, 8)
Score: 0.23698316119183374
(692, 9)
Score: 0.2496930534173079
(692, 10)
Score: 0.21223736793655346


C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWar

(692, 11)
Score: 0.24069853329368293
(692, 12)
Score: 0.2405543572986406
(692, 13)
Score: 0.26652476400375485
(692, 14)
Score: 0.23330119344812736
(692, 15)
Score: 0.21449787927133507
(692, 16)
Score: 0.24929547650708886
(692, 17)
Score: 0.24539963163143427
(692, 18)
Score: 0.23538505048102648
(692, 19)
Score: 0.2191875323653812


C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWar

(692, 20)
Score: 0.26054203846018176
(692, 21)
Score: 0.24329906511465657
(692, 22)
Score: 0.23531695874766237
(692, 23)
Score: 0.2650260514687358
(692, 24)
Score: 0.20699022010426776
(692, 25)
Score: 0.22362775532447277
(692, 26)
Score: 0.25732736364947595
(692, 27)
Score: 0.23173975641157257


C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWar

(692, 28)
Score: 0.2054372661561646
(692, 29)
Score: 0.22988631378382768
(692, 30)
Score: 0.11627105413294951
(692, 31)
Score: 0.1623176539991637
(692, 32)
Score: 0.20293620734634138
(692, 33)
Score: 0.2335553742875544
(692, 34)
Score: 0.2657166452729306
(692, 35)
Score: 0.21427671031857484
(692, 36)
Score: 0.11848503623065645
(692, 37)
Score: 0.19337612496897277
(692, 38)
Score: 0.15557205265333213
(692, 39)
Score: 0.26502960909698675
(692, 40)
Score: 0.26144772927131843
(692, 41)
Score: 0.26137276796504894
(692, 42)
Score: 0.288063052785665
(692, 43)
Score: 0.21941531269069245
(692, 44)
Score: 0.23015897275282918


C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWar

In [8]:
print('O melhor score foi: ', melhor_score)
print('Com as seguintes colunas: \n', melhores_colunas)
len(melhores_colunas)

O melhor score foi:  0.288063052785665
Com as seguintes colunas: 
 Index(['Qtde_Docentes', 'NUM_FUNC', 'AL_CRECHE', 'AL_PRE', '3', '4', '5', '6',
       '6fund', '0A3', '0A4', '4A6', '5A6', '>6', 'TotalEdInf', 'CLEF1S',
       'CLE9F1S', 'CLE9F2S', 'CLE9F3S', 'CLE9F4S', 'CLE9F5S', 'CLE9F6S',
       'CLE9F7S', 'CLE9F8S', 'CLE9F9S', 'ALE9F1S', 'ALE9F2S', 'ALE9F3S',
       'ALE9F4S', 'ALE9F5S', 'ALE9F6S', 'ALE9F7S', 'ALE9F8S', 'ALE9F9S',
       'CLE9F1A5', 'CLE9F6A9', 'ALE9F1A5', 'ALE9F6A9', 'CLE9F1A9', 'ALE9F1A9',
       'CLFUNDI', 'ALFUNDI'],
      dtype='object')


42

### chi 2

In [4]:
#Rodando o selectKBest para 12 colunas
melhor_score = 0
melhores_colunas = []
for var in list(range(1,45)):
    kbest = SelectKBest(chi2, k = var)
    new_data = kbest.fit_transform(data, target)
    print(new_data.shape)
    lm = LinearRegression()
    scaler = StandardScaler()
    data1 = scaler.fit_transform(new_data)
    kf = KFold(5, shuffle = True)
    r = []
    for train_i, test_i in kf.split(data1):
        x_train, x_test = data1[train_i], data1[test_i]
        y_train, y_test = target[train_i], target[test_i]
        lm.fit(x_train, y_train)
        r.append(lm.score(x_test, y_test))
    print('Score:',sum(r)/5)
    if melhor_score < sum(r)/5:
        melhor_score = sum(r)/5
        melhores_colunas = data.columns[kbest.get_support()]

(692, 1)
Score: -0.02025981787327742
(692, 2)
Score: 0.07149222303307359
(692, 3)
Score: 0.09532613609685123
(692, 4)
Score: 0.1711978145101619
(692, 5)
Score: 0.1783385052894731
(692, 6)
Score: 0.16654826666063266
(692, 7)
Score: 0.1921059913593636
(692, 8)
Score: 0.20205367873295582
(692, 9)
Score: 0.191582041421725
(692, 10)
Score: 0.27313795649054917
(692, 11)
Score: 0.2823324173759886
(692, 12)
Score: 0.2738377536732659
(692, 13)
Score: 0.3912984069913136
(692, 14)
Score: 0.3856850263862962
(692, 15)
Score: 0.3413281555678274
(692, 16)
Score: 0.3791356394204407
(692, 17)
Score: 0.36556307293755075
(692, 18)
Score: 0.36071044490828286
(692, 19)
Score: 0.313371626501013
(692, 20)
Score: 0.3768397115657617
(692, 21)
Score: 0.3585347798699667
(692, 22)
Score: 0.359310641806626
(692, 23)
Score: 0.36914576763785323
(692, 24)
Score: 0.32985639539168154
(692, 25)
Score: 0.38736918565071077
(692, 26)
Score: 0.3740017505907332
(692, 27)
Score: 0.3939731159429688
(692, 28)
Score: 0.360571266

In [5]:
print('O melhor score foi: ', melhor_score)
print('Com as seguintes colunas: \n', melhores_colunas)

O melhor score foi:  0.4469624440321357
Com as seguintes colunas: 
 Index(['Qtde_Docentes', 'NUM_FUNC', 'SETEDU', 'AL_PRE', '5', '6', '6fund',
       '0A4', '4A6', '5A6', 'TotalEdInf', 'ALEM1S', 'ALEM2S', 'ALEM3S', 'ALEM',
       'ALEF3S', 'ALEF4S', 'ALEF5S', 'ALEF6S', 'ALEF7S', 'ALEF8S', 'ALEF1A4',
       'ALEF5A8', 'ALEF1A8', 'ALE9F8S', 'ALE9F9S', 'ALE9F1A5', 'ALE9F6A9',
       'ALE9F1A9', 'ALFUNDI', 'ALFUNDII', 'ALFUND', 'ALEJAMED', 'TOTALEJA',
       'ALEDPROF', 'TOTAL_ALUNOS'],
      dtype='object')


In [60]:
len(melhores_colunas)

35